## Project Final Report

DSCI 100, Section 9，Group 38   
Anson Chen, Amber O'Neile, Nathania So, Xiaoran Sun

### Introduction

The data being analyzed in this project was collected from a Minecraft server run by a group of Computer Science students at UBC$^{[1]}$. The group collected both demographic and behavioural data about the players for the purposes of training artificial intelligence$^{[1]}$.

In this project, we seek to answer the question: **Can player age and gameplay time be used to predict if players will subscribe to the CS group's game-related newsletter?**

The players dataset contains demographical information about 196 Minecraft players in 196 observations and 7 variables. This information was collected through a form for participants in a study by Frank Wood $^{[2]}$. The variables in this dataset, and summary statistics regarding them when applicable, are listed below.

| Variable     | Type      | Meaning                                                   | Mean  | Median |
| ------------ | --------- | --------------------------------------------------------- | ----- | ------ |
| experience   | factor    | Experience level of the player                            | NA    | NA     |
| subscribe    | logical   | Whether the player is subscribed to the newsletter or not | NA    | NA     |
| hashedEmail  | character | Encoded email addresses of players to maintain privacy    | NA    | NA     |
| played-hours | double    | Total hours of Minecraft played                           | 5.85  | 0.1    |
| name         | character | Name of players                                           | NA    | NA     |
| gender       | factor    | Gender of the player                                      | NA    | NA     | 
| age          | integer   | Age of the player in years                                | 21.14 | 19     |

### Methods and Results

#### Exploratory Analysis and Visualization

First, we import the players.csv into R.
The players dataset provides demographic information such as player age. The file is loaded using read_csv( ).

In [ ]:
library(tidyverse)
library(tidymodels)
library(cowplot)
url_1 = 'https://drive.google.com/uc?export=download&id=1hSH3Uxef4JcuooVcPXiY0wfRlW0Qahbs'
players <- read_csv(url_1)
dim(players); head(players)

In [ ]:
# Creating a bar graph for age
options(repr.plot.width = 13, repr.plot.height = 6)
age_plot <- players |>
            ggplot(aes(x=Age, fill=subscribe)) +
            geom_bar(position = "stack") +
            scale_fill_discrete(name="Subscription Status", labels = c("No", "Yes")) + #Function learned from source [3] in references
            labs(x="Age (years)", y="Number of Players", title = "Figure 1 - Number of Players per Subscription Status by Age") +
            scale_x_continuous(breaks = seq(0, 60, by =2)) + # Function learned from source [4] in references 
            theme(plot.title = element_text(size=16, face="bold", hjust = 0.5), axis.title = element_text(size = 13), 
            legend.title = element_text(size = 16), axis.text = element_text(size=13), legend.text = element_text(size = 13))
age_plot

This plot shows mostly 17 year-old players, indicating an uneven distribution that can skew results. Players aged 30-60 have a higher proportion of non-subscribers, while younger players subscribe more.

In [ ]:
# Creating a  graph for length of time played
time_plot <- players |>
            ggplot(aes(x=played_hours, fill=subscribe)) +
            geom_histogram(position = "stack", binwidth = 2) +
            scale_fill_discrete(name="Subscription Status", labels = c("No", "Yes")) + # [3]
            scale_x_continuous(breaks = seq(0, 223.1, by =10)) + # [4]
            labs(x="Total Hours of Gameplay", y="Number of Players", title = "Figure 2 - Number of Players per Subscription Status by Hours of Gameplay") +
            theme(plot.title = element_text(size=16, face="bold", hjust = 0.5), axis.title = element_text(size = 13), 
            legend.title = element_text(size = 16), axis.text = element_text(size=13), legend.text = element_text(size = 13))
time_plot

This histogram shows that most play 0-5 hours, with a right-skewed distribution. This means predictions for high-hour players rely on limited data and are less reliable.

In [ ]:
# Creating a  graph for all variables
log_hours_players <- players |>
            mutate(log_hours = log10(played_hours))

all_var_scatter <- log_hours_players |>
            ggplot(aes(x=Age, y=log_hours, color=subscribe)) +
            geom_point(size = 2, alpha = 0.8) +
            scale_colour_discrete(name="Subscription Status", labels = c("No", "Yes")) + # [3]
            scale_x_continuous(breaks = seq(0, 60, by =10)) + # [4]
            labs(x="Age of Player", y="Hours Played (log scale)", title = "Figure 3 - Log-Transformed Hours Played vs Age of Player") +
            theme(plot.title = element_text(size=16, face="bold", hjust = 0.5), axis.title = element_text(size = 13), 
            legend.title = element_text(size = 16), axis.text = element_text(size=13), legend.text = element_text(size = 13))
all_var_scatter

The scatterplot shows that players aged 15-25 have the most gameplay hours, with maximum hours generally decreasing as age increases (a negative correlation). A logarithmic y-axis is used to better display low-value points. There is no clear relationship between age and subscription status. However, all players with 10+ hours of gameplay are subscribed to newsletters.

#### Data Analysis

To investigate which player characteristics are predictive of subscribing to game-related newsletters, we selected age and total play hours as our key variables. Our analysis follows a complete data-science workflow, from cleaning the data to preparing it for modeling. Below, we describe the steps we used to process and wrangle the data before analysis. We will use a K-Nearest Neighbours (KNN) classification model to predict whether a player subscribes to the newsletter. Our response variable (y) is the subscription status, and our predictors (x) are age and played hours.
KNN is chosen because the outcome is a binary categorical variable and the relationship between age, played hours, and subscription is unlikely to be perfectly linear. Unlike linear regression, which requires a continuous outcome and strong assumptions about linearity, KNN is a flexible non-parametric method that can capture more complex patterns. Since KNN relies on distance calculations, we will standardize age and average hours played so that both variables contribute equally to the model.

1. Wrangle and Clean Data:   
To begin, we cleaned the merged dataset by removing rows where essential information such as age, play sessions was missing or unusable.

In [ ]:
# cleaned_data <- combined_data |>
cleaned_data <- players |>
  filter(!is.na(Age),
         !is.na(played_hours))

Select relevant data only

In [ ]:
cleaned_data <- cleaned_data |> select(Age, played_hours, subscribe)

Then we convert "subscribe" from logical to factor form.

In [ ]:
cleaned_data <- cleaned_data|> mutate(subscribe = as_factor(subscribe))

2. Split the Data: 
   Split the data set into 75% for training, reserving 25% for testing, then apply 5-fold cross-validation to the training set

In [ ]:
set.seed(1357)
data_split <- initial_split(cleaned_data, prop = 0.75, strata = subscribe)  
data_train <- training(data_split)   
data_test <- testing(data_split)

fold <- vfold_cv(data_train, v=5, strata = subscribe)


3. Determining the best number of neighbours

In [ ]:
knn_recipe <- recipe(subscribe ~ Age + played_hours, data = data_train) |>
            step_center(all_predictors()) |>
            step_scale(all_predictors())

knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
        set_engine("kknn") |>
        set_mode("classification")
k_vals <- tibble(neighbors = seq(from = 1, to = 20, by = 1))

results <- workflow() |>
      add_recipe(knn_recipe) |>
      add_model(knn_tune) |>
      tune_grid(resamples = fold, grid = k_vals) |>
      collect_metrics()

accuracies <- results |>
            filter(.metric == "accuracy")

cross_val_plot <- accuracies |>
                ggplot(aes(x=neighbors, y=mean)) +
                geom_point() + geom_line() +
                labs(x = "Number of Neighbours (k)", y= "Accuracy of Estimate", title = "Figure 4 - Accuracy per Number of Neighbours") +
                theme(plot.title = element_text(size=16, face="bold", hjust = 0.5), 
                axis.title = element_text(size = 13), axis.text = element_text(size=13))
cross_val_plot

The chosen number of neighbours is 18 because that provides the highest accuracy.

4. Make a new specification based on the best number of neighbours found above

In [ ]:
knn_spec <- nearest_neighbor(weight_func="rectangular", neighbors=18) |>
            set_engine("kknn") |>
            set_mode("classification")

5. Put it all together in a workflow and fit the model to the training data

In [ ]:
knn_fit <- workflow() |>
          add_recipe(knn_recipe) |>
          add_model(knn_spec) |>
          fit(data = data_train)

6. Fit the model to the test data and then run a few metrics to see the performance of the model

In [ ]:
subscription_predictions <- predict(knn_fit, data_test) |>
                    bind_cols(data_test)

subscription_accuracy <- subscription_predictions |>
                metrics(truth = subscribe, estimate = .pred_class) |>
                filter(.metric == "accuracy")
subscription_precision <- subscription_predictions |>
                precision(truth = subscribe, estimate = .pred_class, event_level = "second") # [5]
subscription_recall <- subscription_predictions |>
                recall(truth = subscribe, estimate = .pred_class, event_level = "second") # [5]
subscription_conf_mat <- subscription_predictions |>
                conf_mat(truth = subscribe, estimate = .pred_class)
subscription_accuracy
subscription_precision
subscription_recall
subscription_conf_mat

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 5)

#plot of analysis results
analysis_plot <- subscription_predictions |>
    ggplot(aes(x = Age, y = played_hours)) +
    scale_colour_discrete(name="Subscription Status", labels = c("No", "Yes")) + # [3]
    geom_point(aes(color = .pred_class)) +
    labs(x = 'Age', y = 'Played Hours', color= 'Pred Class', title = "Figure 5 - Predicted Subscription Status") +
    theme(plot.title = element_text(size=16, face="bold", hjust = 0.5), axis.title = element_text(size = 13), 
    legend.title = element_text(size = 16), axis.text = element_text(size=13), legend.text = element_text(size = 13))
analysis_plot

options(repr.plot.width = 12.5, repr.plot.height = 5)
#plot of true classification
real_plot <- data_test |>
    ggplot(aes(x = Age, y = played_hours)) +
    scale_colour_discrete(name="True Subscription Status", labels = c("No", "Yes")) + # [3]
    geom_point(aes(color = subscribe)) +
    labs(x = 'Age', y = 'Played Hours', color= 'subscribe', title = "Figure 6 - True Subscription Status") +
    theme(plot.title = element_text(size=16, face="bold", hjust = 0.5), axis.title = element_text(size = 13), 
    legend.title = element_text(size = 16), axis.text = element_text(size=13), legend.text = element_text(size = 13))
real_plot


This graph is a visual representation of the predictions of subscription status for the test set.

### Discussion

This classification model was designed to predict whether a player will subscribe to a game based newspaper based on their age and hours played. Five-fold cross validation with 18 nearest neighbours in the K-nearest neighbours model provided the best performance on training data, so it was chosen for the final model. From the visualizations above comparing the predicted and true subscription status, it is clear that there are some statuses predicted incorrectly, suggesting that age and hours played may not be the best predictors.

The model had 71.4% accuracy when assessing the test data with a precision of 73.9%, and a recall of 94.4%. This indicates that the model does not frequently miss detecting potential subscribers (high recall), but also has a notable number of false positives (in this case predicting a player will subscribe when they will not, leading to moderate precision). Because the model mislabelled many non-subscribers as subscribers, the overall accuracy was lower.

These results indicate that while age and played hours provided some insight into whether players will subscribe to the newsletter or not, the metrics suggest that these predictors alone cannot reliable identify subscribers from non-subscribers. Subscription status is likely influenced by a combination of additional variables rather than strictly age and hours played.

This lines up initial expectations. The exploratory visualizations (**Figures 1**, **2** and **3**) showed some level of correlation between age, hours played and subscription status, but none that immediately stood out as strong. It was expected that a certain level of predictive accuracy could be found by using these variables, but since no trends were especially clear in the explorations, it is doubtful that accuracy would be particularly high.

In **Figure 1**, it was noticed that all players with aged 16 and under were subscribed to the newsletter. This indicated that there might have been some sort of trend there, but since the vast majority of players were 17 years of age or older, one could not be fully confident that the apparent trend would result to much. Something similar with **Figure 2** was noticed. All players with over 10 hours played were subscribed to the newsletter, but almost all players had played under 10 hours. **Figure 3** was a combination of the two previous visualizations, and thus preserved the trends noticed before, but also didn't offer many new insights.

This led to middling expectations about the efficiency of the model, which were reflected in reality once the data was analyzed and predictions made.

Given that age and hours played alone did not produce strong predictive power, several future directions emerge. First, additional predictors could be incorporated to capture behavioural or motivational factors that age and play time cannot explain. For instance, player experience level may better reflect familiarity with the platform and the likelihood of engaging with newsletters. Other potentially important variables include gender, session frequency, or social activity indicators (e.g., number of friends, participation in events), which may be more directly tied to engagement.

Future work could also explore different combinations of predictors or interactions among variables, as subscription behaviour is likely influenced by multiple factors simultaneously rather than single features in isolation. Finally, evaluating alternative modelling approaches such as logistic regression or tree-based models may help uncover non-linear patterns that KNN cannot capture. These directions would help determine which features or model structures more effectively distinguish subscribers from non-subscribers.

### References

[1] Pacific Laboratory of Artificial Intelligence General  Information. (2024, March 19). PLAICraft. https://plaicraft.ai/faq/general   
[2] Pacific Laboratory of Artifical Intelligence. (2024, March 19). Plaicraft. PLAICraft. https://plaicraft.ai/   
[3] How do I manually change the key labels in a legend in GGPLOT2. (2017, October 27). Stack Overflow. https://stackoverflow.com/questions/7323191/how-do-i-manually-change-the-key-labels-in-a-legend-in-ggplot2   
[4] GGPLOT2 axis titles, labels, ticks, limits and scales. (2022, December 9). R CHARTS. https://r-charts.com/ggplot2/axis/#google_vignette   
[5] Kuhn, M., & Vaughan, D. (2021, March 28). Yardstick: Precision – R documentation. Quantargo. https://www.quantargo.com/help/r/latest/packages/yardstick/0.0.8/precision